In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, accuracy_score
# --- CORRECTED IMPORTS: Removed TFTrainer and TFTrainingArguments ---
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [ ]:
# --- Configuration ---
MODEL_NAME = "google/muril-base-cased"
TEXT_COL = "text_no_stopwords"
LABEL_COLS = ['defamation', 'hate', 'non-hate', 'violence', 'vulgar']
MAX_LENGTH = 128
NUM_LABELS = len(LABEL_COLS)

In [ ]:
# --- 1. Load and Prepare Data ---
# Assuming the file "hate_speech_hindi_final.csv" is available
df = pd.read_csv("/content/hate_speech_hindi_final.csv")

# Create the label array (y_true)
y = df[LABEL_COLS].values

# Split the data
X_train, X_val, y_train, y_val = train_test_split(
    df[TEXT_COL].tolist(),
    y,
    test_size=0.2,
    random_state=42
)

In [ ]:
# --- 2. Load Tokenizer and Model ---
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = TFAutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    problem_type="multi_label_classification"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier', 'bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# --- 3. Tokenization ---
val_encodings = tokenizer(
    X_val,
    truncation=True,
    padding=True,
    max_length=MAX_LENGTH,
    return_tensors="tf"
)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


In [ ]:
# Create TensorFlow Dataset for evaluation
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_val
)).batch(16)

In [ ]:
# --- 4. Model Compile and Prediction ---

# Using standard Keras compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

print("--- Running prediction on validation data... ---")

# Run prediction to get raw logits
pred_outputs = model.predict(val_dataset)
logits = pred_outputs.logits if hasattr(pred_outputs, "logits") else pred_outputs
probs = tf.sigmoid(logits).numpy()
y_true = y_val

print("Raw logits generated. Proceeding to optimized evaluation.")
print("-" * 30)

--- Running prediction on validation data... ---
499/499 [==============================] - 72s 128ms/step
Raw logits generated. Proceeding to optimized evaluation.
------------------------------


In [ ]:
# --- 5. Optimized Evaluation ---

# A crucial step for multi-label F1 score: find the best threshold.
print("## 🧪 Threshold Optimization...")

best_threshold = 0.5
best_macro_f1 = 0.0

# Test thresholds from 0.05 to 0.50
for threshold in np.arange(0.05, 0.55, 0.05):
    y_pred_tuned = (probs >= threshold).astype(int)
    macro_f1 = f1_score(y_true, y_pred_tuned, average="macro", zero_division=0)

    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
        best_threshold = threshold

print(f"Optimal Threshold found: {best_threshold:.2f} (Macro F1: {best_macro_f1:.4f})")
print("-" * 30)

# Use the best threshold found for final metrics
y_pred = (probs >= best_threshold).astype(int)

print("\n--- Final Evaluation with Optimized Threshold ---")

# Exact Match Accuracy (Subset Accuracy)
subset_accuracy = accuracy_score(y_true, y_pred)
macro_f1 = f1_score(y_true, y_pred, average="macro", zero_division=0)
micro_f1 = f1_score(y_true, y_pred, average="micro", zero_division=0)

print(f"\nExact Match Accuracy: {subset_accuracy:.4f}")
print(f"Macro F1 Score: {macro_f1:.4f}")
print(f"Micro F1 Score: {micro_f1:.4f}")

print("\nClassification Report:\n")
print(classification_report(
    y_true,
    y_pred,
    target_names=LABEL_COLS,
    zero_division=0
))

## 🧪 Threshold Optimization...
Optimal Threshold found: 0.05 (Macro F1: 0.4006)
------------------------------

--- Final Evaluation with Optimized Threshold ---

Exact Match Accuracy: 0.0000
Macro F1 Score: 0.4006
Micro F1 Score: 0.4006

Classification Report:

              precision    recall  f1-score   support

  defamation       0.25      1.00      0.39      1954
        hate       0.25      1.00      0.40      1994
    non-hate       0.25      1.00      0.40      2021
    violence       0.25      1.00      0.40      2017
      vulgar       0.25      1.00      0.40      1998

   micro avg       0.25      1.00      0.40      9984
   macro avg       0.25      1.00      0.40      9984
weighted avg       0.25      1.00      0.40      9984
 samples avg       0.25      1.00      0.39      9984



In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, accuracy_score
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [ ]:
# --- Configuration ---
MODEL_NAME = "google/muril-base-cased"
TEXT_COL = "text_no_stopwords"
LABEL_COLS = ['defamation', 'hate', 'non-hate', 'violence', 'vulgar']
MAX_LENGTH = 128
NUM_LABELS = len(LABEL_COLS)
LEARNING_RATE = 5e-5 # Standard fine-tuning rate
EPOCHS = 5 # Recommended 2-4 epochs for fine-tuning
BATCH_SIZE = 16 # Use small batches to avoid memory issues


In [ ]:
# --- 1. Load and Prepare Data ---
df = pd.read_csv("hate_speech_hindi_final.csv")
y = df[LABEL_COLS].values
X_train, X_val, y_train, y_val = train_test_split(
    df[TEXT_COL].tolist(),
    y,
    test_size=0.2,
    random_state=42
)

In [ ]:
# --- 2. Load Tokenizer and Model ---
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = TFAutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    problem_type="multi_label_classification"
)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier', 'bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# --- 3. Tokenization and Dataset Creation ---
# Tokenize Training Data
train_encodings = tokenizer(
    X_train,
    truncation=True,
    padding=True,
    max_length=MAX_LENGTH,
    return_tensors="tf"
)
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
)).shuffle(100).batch(BATCH_SIZE)

# Tokenize Validation Data
val_encodings = tokenizer(
    X_val,
    truncation=True,
    padding=True,
    max_length=MAX_LENGTH,
    return_tensors="tf"
)
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_val
)).batch(BATCH_SIZE)

In [ ]:
# --- 4. Compile and Train the Model ---

# We compile the model using BinaryCrossentropy(from_logits=True)
# which applies sigmoid internally and is correct for multi-label tasks.
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

print(f"--- Starting MuRIL Fine-Tuning for {EPOCHS} Epochs ---")
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset
)
print("--- Training Complete ---")
print("-" * 50)

--- Starting MuRIL Fine-Tuning for 5 Epochs ---
Epoch 1/5
1994/1994 [==============================] - 947s 446ms/step - loss: 0.5667 - accuracy: 0.2009 - val_loss: 0.5638 - val_accuracy: 0.1844
Epoch 2/5
1994/1994 [==============================] - 873s 438ms/step - loss: 0.5636 - accuracy: 0.2004 - val_loss: 0.5637 - val_accuracy: 0.1846
Epoch 3/5
1994/1994 [==============================] - 872s 437ms/step - loss: 0.5637 - accuracy: 0.1986 - val_loss: 0.5636 - val_accuracy: 0.1846
Epoch 4/5
1994/1994 [==============================] - 871s 437ms/step - loss: 0.5637 - accuracy: 0.2016 - val_loss: 0.5637 - val_accuracy: 0.1841
Epoch 5/5
1994/1994 [==============================] - 871s 437ms/step - loss: 0.5637 - accuracy: 0.2006 - val_loss: 0.5635 - val_accuracy: 0.1841
--- Training Complete ---
--------------------------------------------------


In [ ]:
# --- 5. Prediction and Optimized Evaluation ---

print("--- Generating Predictions on Validation Data ---")
pred_outputs = model.predict(val_dataset)
logits = pred_outputs.logits
probs = tf.sigmoid(logits).numpy()
y_true = y_val

# Threshold Optimization
print("\n## 🧪 Threshold Optimization...")
best_threshold = 0.5
best_macro_f1 = 0.0

for threshold in np.arange(0.05, 0.55, 0.05):
    y_pred_tuned = (probs >= threshold).astype(int)
    macro_f1 = f1_score(y_true, y_pred_tuned, average="macro", zero_division=0)

    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
        best_threshold = threshold

print(f"Optimal Threshold found: {best_threshold:.2f} (Macro F1: {best_macro_f1:.4f})")
print("-" * 50)

# Final Metrics
y_pred = (probs >= best_threshold).astype(int)
subset_accuracy = accuracy_score(y_true, y_pred)
macro_f1 = f1_score(y_true, y_pred, average="macro", zero_division=0)
micro_f1 = f1_score(y_true, y_pred, average="micro", zero_division=0)

print("\n--- Final Evaluation with Optimized Threshold ---")
print(f"Exact Match Accuracy: {subset_accuracy:.4f}")
print(f"Macro F1 Score: {macro_f1:.4f}")
print(f"Micro F1 Score: {micro_f1:.4f}")

print("\nClassification Report:\n")
print(classification_report(
    y_true,
    y_pred,
    target_names=LABEL_COLS,
    zero_division=0
))

--- Generating Predictions on Validation Data ---
499/499 [==============================] - 64s 120ms/step

## 🧪 Threshold Optimization...
Optimal Threshold found: 0.05 (Macro F1: 0.4006)
--------------------------------------------------

--- Final Evaluation with Optimized Threshold ---
Exact Match Accuracy: 0.0000
Macro F1 Score: 0.4006
Micro F1 Score: 0.4006

Classification Report:

              precision    recall  f1-score   support

  defamation       0.25      1.00      0.39      1954
        hate       0.25      1.00      0.40      1994
    non-hate       0.25      1.00      0.40      2021
    violence       0.25      1.00      0.40      2017
      vulgar       0.25      1.00      0.40      1998

   micro avg       0.25      1.00      0.40      9984
   macro avg       0.25      1.00      0.40      9984
weighted avg       0.25      1.00      0.40      9984
 samples avg       0.25      1.00      0.39      9984



In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, accuracy_score
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# --- Configuration (UPDATED) ---
MODEL_NAME = "google/muril-base-cased"
TEXT_COL = "text_no_stopwords"
LABEL_COLS = ['defamation', 'hate', 'non-hate', 'violence', 'vulgar']
MAX_LENGTH = 128
NUM_LABELS = len(LABEL_COLS)
LEARNING_RATE = 1e-5 # ***LOWERED LEARNING RATE: 5e-5 -> 1e-5***
EPOCHS = 1 # ***Reduced to 1 epoch for quick test of new LR***
BATCH_SIZE = 16

# --- 1. Load and Prepare Data ---
df = pd.read_csv("hate_speech_hindi_final.csv")
y = df[LABEL_COLS].values
X_train, X_val, y_train, y_val = train_test_split(
    df[TEXT_COL].tolist(),
    y,
    test_size=0.2,
    random_state=42
)

# --- 2. Load Tokenizer and Model ---
print(f"Reloading tokenizer and model: {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# Model weights are re-initialized here
model = TFAutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    problem_type="multi_label_classification"
)

# --- 3. Tokenization and Dataset Creation ---
print("Tokenizing data and creating TF datasets...")
# Tokenize Training Data
train_encodings = tokenizer(
    X_train,
    truncation=True,
    padding=True,
    max_length=MAX_LENGTH,
    return_tensors="tf"
)
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
)).shuffle(100).batch(BATCH_SIZE)

# Tokenize Validation Data
val_encodings = tokenizer(
    X_val,
    truncation=True,
    padding=True,
    max_length=MAX_LENGTH,
    return_tensors="tf"
)
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_val
)).batch(BATCH_SIZE)


# --- 4. Compile and Train the Model (1 Epoch) ---
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

print("-" * 50)
print(f"--- Starting NEW MuRIL Fine-Tuning for {EPOCHS} Epoch (NEW LR: {LEARNING_RATE}) ---")
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset
)
print("--- Training Complete ---")
print("-" * 50)


# --- 5. Prediction and Optimized Evaluation ---
print("--- Generating Predictions on Validation Data ---")
pred_outputs = model.predict(val_dataset)
logits = pred_outputs.logits
probs = tf.sigmoid(logits).numpy()
y_true = y_val

# Threshold Optimization
print("\n## 🧪 Threshold Optimization...")
best_threshold = 0.5
best_macro_f1 = 0.0

# Iterating to find the best threshold for Macro F1
for threshold in np.arange(0.05, 0.55, 0.05):
    y_pred_tuned = (probs >= threshold).astype(int)
    macro_f1 = f1_score(y_true, y_pred_tuned, average="macro", zero_division=0)

    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
        best_threshold = threshold

print(f"Optimal Threshold found: {best_threshold:.2f} (Macro F1: {best_macro_f1:.4f})")
print("-" * 50)

# Final Metrics
y_pred = (probs >= best_threshold).astype(int)
subset_accuracy = accuracy_score(y_true, y_pred)
macro_f1_final = f1_score(y_true, y_pred, average="macro", zero_division=0)
micro_f1_final = f1_score(y_true, y_pred, average="micro", zero_division=0)

print("\n--- Final Evaluation with Optimized Threshold ---")
print(f"Exact Match Accuracy: {subset_accuracy:.4f}")
print(f"Macro F1 Score: {macro_f1_final:.4f}")
print(f"Micro F1 Score: {micro_f1_final:.4f}")

print("\nClassification Report:\n")
print(classification_report(
    y_true,
    y_pred,
    target_names=LABEL_COLS,
    zero_division=0
))

Reloading tokenizer and model: google/muril-base-cased...


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier', 'bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenizing data and creating TF datasets...
--------------------------------------------------
--- Starting NEW MuRIL Fine-Tuning for 1 Epoch (NEW LR: 1e-05) ---
1994/1994 [==============================] - 937s 446ms/step - loss: 0.5565 - accuracy: 0.2985 - val_loss: 0.4948 - val_accuracy: 0.3947
--- Training Complete ---
--------------------------------------------------
--- Generating Predictions on Validation Data ---
499/499 [==============================] - 61s 118ms/step

## 🧪 Threshold Optimization...
Optimal Threshold found: 0.25 (Macro F1: 0.5159)
--------------------------------------------------

--- Final Evaluation with Optimized Threshold ---
Exact Match Accuracy: 0.0946
Macro F1 Score: 0.5159
Micro F1 Score: 0.5200

Classification Report:

              precision    recall  f1-score   support

  defamation       0.32      0.79      0.45      1954
        hate       0.36      0.47      0.41      1994
    non-hate       0.42      0.86      0.56      2021
    violence    

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, classification_report, accuracy_score

# --- Assuming 'probs' (the sigmoid output) and 'y_true' (the true labels)
# from the previous execution step are in memory. ---

LABEL_COLS = ['defamation', 'hate', 'non-hate', 'violence', 'vulgar']
best_thresholds = {}
NUM_LABELS = len(LABEL_COLS)

print("\n## 🎯 Per-Class Threshold Optimization...")
print("-" * 50)

# Iterate through each of the 5 classes (columns)
for i, col_name in enumerate(LABEL_COLS):

    col_probs = probs[:, i]     # Probabilities for this class
    col_y_true = y_true[:, i]   # True labels for this class

    best_f1 = -1
    best_t = 0.5

    # Test thresholds from 0.05 to 0.50
    for t in np.arange(0.05, 0.55, 0.05):
        y_pred_t = (col_probs >= t).astype(int)

        # Calculate F1-score for this single class
        f1 = f1_score(col_y_true, y_pred_t, zero_division=0)

        if f1 > best_f1:
            best_f1 = f1
            best_t = t

    best_thresholds[col_name] = best_t
    print(f"Class '{col_name}': Optimal Threshold = {best_t:.2f} (F1: {best_f1:.4f})")

# --- Final Evaluation using ALL Optimal Thresholds ---

y_pred_final_multi = np.zeros_like(y_true, dtype=int)

# Apply each class's best threshold to its respective column
for i, col_name in enumerate(LABEL_COLS):
    t = best_thresholds[col_name]
    y_pred_final_multi[:, i] = (probs[:, i] >= t).astype(int)


print("\n--- Final Evaluation with Per-Class Optimized Thresholds ---")

subset_accuracy = accuracy_score(y_true, y_pred_final_multi)
macro_f1 = f1_score(y_true, y_pred_final_multi, average="macro", zero_division=0)
micro_f1 = f1_score(y_true, y_pred_final_multi, average="micro", zero_division=0)

print(f"\nExact Match Accuracy: {subset_accuracy:.4f}")
print(f"Macro F1 Score: {macro_f1:.4f}")
print(f"Micro F1 Score: {micro_f1:.4f}")

print("\nClassification Report:\n")
print(classification_report(
    y_true,
    y_pred_final_multi,
    target_names=LABEL_COLS,
    zero_division=0
))


## 🎯 Per-Class Threshold Optimization...
--------------------------------------------------
Class 'defamation': Optimal Threshold = 0.30 (F1: 0.4570)
Class 'hate': Optimal Threshold = 0.20 (F1: 0.4217)
Class 'non-hate': Optimal Threshold = 0.40 (F1: 0.5797)
Class 'violence': Optimal Threshold = 0.30 (F1: 0.6048)
Class 'vulgar': Optimal Threshold = 0.40 (F1: 0.6074)

--- Final Evaluation with Per-Class Optimized Thresholds ---

Exact Match Accuracy: 0.0916
Macro F1 Score: 0.5341
Micro F1 Score: 0.5133

Classification Report:

              precision    recall  f1-score   support

  defamation       0.33      0.75      0.46      1954
        hate       0.27      0.98      0.42      1994
    non-hate       0.55      0.62      0.58      2021
    violence       0.52      0.73      0.60      2017
      vulgar       0.50      0.78      0.61      1998

   micro avg       0.38      0.77      0.51      9984
   macro avg       0.43      0.77      0.53      9984
weighted avg       0.43      0.77 

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, accuracy_score
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# --- Configuration ---
MODEL_NAME = "google/muril-base-cased"
TEXT_COL = "text_no_stopwords"
LABEL_COLS = ['defamation', 'hate', 'non-hate', 'violence', 'vulgar']
MAX_LENGTH = 128
NUM_LABELS = len(LABEL_COLS)
LEARNING_RATE = 1e-5
EPOCHS = 2
BATCH_SIZE = 16

# --- 1. Load and Prepare Data ---
df = pd.read_csv("hate_speech_hindi_final.csv")
y = df[LABEL_COLS].values
X_train, X_val, y_train, y_val = train_test_split(
    df[TEXT_COL].tolist(),
    y,
    test_size=0.2,
    random_state=42
)

# --- 2. Calculate Sample Weights (The Fix) ---

# Calculate class-wise weights (5D array) as before
pos_counts = y_train.sum(axis=0)
neg_counts = len(y_train) - pos_counts
total = pos_counts + neg_counts
pos_weights = (1 / pos_counts) * (total / 2.0)
neg_weights = (1 / neg_counts) * (total / 2.0)

sample_weight_matrix_5d = np.zeros_like(y_train, dtype=np.float32)

for i in range(NUM_LABELS):
    sample_weight_matrix_5d[:, i][y_train[:, i] == 1] = pos_weights[i]
    sample_weight_matrix_5d[:, i][y_train[:, i] == 0] = neg_weights[i]

# *** FIX: Convert 5D weight matrix to a 1D per-sample weight vector ***
# This averages the 5 weights for each sample, satisfying the Keras shape requirement.
sample_weights_1d = sample_weight_matrix_5d.mean(axis=1)


# --- 3. Load Tokenizer and Model ---
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = TFAutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    problem_type="multi_label_classification"
)

# --- 4. Tokenization and Dataset Creation (Using 1D weight in dataset) ---
train_encodings = tokenizer(
    X_train,
    truncation=True,
    padding=True,
    max_length=MAX_LENGTH,
    return_tensors="tf"
)
# Train dataset now includes the 1D weight as the 3rd element
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train,
    sample_weights_1d # Passing 1D weight here
)).shuffle(100).batch(BATCH_SIZE)

# Validation Dataset
val_encodings = tokenizer(
    X_val,
    truncation=True,
    padding=True,
    max_length=MAX_LENGTH,
    return_tensors="tf"
)
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_val
)).batch(BATCH_SIZE)


# --- 5. Compile and Train the Model (Standard BCE Loss) ---
# Revert to standard BCE loss since sample_weight is now 1D
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

print("-" * 50)
print(f"--- Continuing Fine-Tuning for {EPOCHS} Epochs with Per-Sample Averaged Weighting ---")
# Keras will automatically pull the 1D sample_weight from train_dataset
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset
)
print("--- Training Complete ---")
print("-" * 50)


# --- 6. Prediction and Per-Class Optimized Evaluation ---
print("--- Generating Predictions on Validation Data ---")
pred_outputs = model.predict(val_dataset)
logits = pred_outputs.logits
probs = tf.sigmoid(logits).numpy()
y_true = y_val

# Threshold Optimization
print("\n## 🧪 Threshold Optimization...")
best_thresholds = {}

for i, col_name in enumerate(LABEL_COLS):
    col_probs = probs[:, i]
    col_y_true = y_true[:, i]
    best_f1 = -1
    best_t = 0.5

    # Test a fine-grained threshold range
    for t in np.arange(0.01, 0.61, 0.02):
        y_pred_t = (col_probs >= t).astype(int)
        f1 = f1_score(col_y_true, y_pred_t, zero_division=0)

        if f1 > best_f1:
            best_f1 = f1
            best_t = t

    best_thresholds[col_name] = best_t
    print(f"Class '{col_name}': Optimal Threshold = {best_t:.2f} (F1: {best_f1:.4f})")

# Final Metrics
y_pred_final_multi = np.zeros_like(y_true, dtype=int)
for i, col_name in enumerate(LABEL_COLS):
    t = best_thresholds[col_name]
    y_pred_final_multi[:, i] = (probs[:, i] >= t).astype(int)

subset_accuracy = accuracy_score(y_true, y_pred_final_multi)
macro_f1_final = f1_score(y_true, y_pred_final_multi, average="macro", zero_division=0)
micro_f1_final = f1_score(y_true, y_pred_final_multi, average="micro", zero_division=0)

print("\n--- Final Evaluation with Per-Class Optimized Thresholds ---")
print(f"Exact Match Accuracy: {subset_accuracy:.4f}")
print(f"Macro F1 Score: {macro_f1_final:.4f}")
print(f"Micro F1 Score: {micro_f1_final:.4f}")

print("\nClassification Report:\n")
print(classification_report(
    y_true,
    y_pred_final_multi,
    target_names=LABEL_COLS,
    zero_division=0
))

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier', 'bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


--------------------------------------------------
--- Continuing Fine-Tuning for 2 Epochs with Per-Sample Averaged Weighting ---
Epoch 1/2
1994/1994 [==============================] - 937s 446ms/step - loss: 0.5713 - accuracy: 0.2879 - val_loss: 0.4938 - val_accuracy: 0.4086
Epoch 2/2
1994/1994 [==============================] - 875s 439ms/step - loss: 0.4847 - accuracy: 0.4247 - val_loss: 0.4465 - val_accuracy: 0.4954
--- Training Complete ---
--------------------------------------------------
--- Generating Predictions on Validation Data ---
499/499 [==============================] - 62s 118ms/step

## 🧪 Threshold Optimization...
Class 'defamation': Optimal Threshold = 0.31 (F1: 0.4780)
Class 'hate': Optimal Threshold = 0.25 (F1: 0.4913)
Class 'non-hate': Optimal Threshold = 0.37 (F1: 0.6587)
Class 'violence': Optimal Threshold = 0.41 (F1: 0.6511)
Class 'vulgar': Optimal Threshold = 0.51 (F1: 0.6881)

--- Final Evaluation with Per-Class Optimized Thresholds ---
Exact Match Accuracy:

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, accuracy_score
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# --- Custom Loss Function ---
def WeightedBinaryCrossentropy(y_true, y_pred, sample_weight=None):
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
    loss = bce(y_true, y_pred)
    if sample_weight is not None:
        loss = loss * sample_weight
    return tf.reduce_mean(loss)


# --- Configuration (UPDATED) ---
MODEL_NAME = "google/muril-base-cased"
TEXT_COL = "text_no_stopwords"
LABEL_COLS = ['defamation', 'hate', 'non-hate', 'violence', 'vulgar']
MAX_LENGTH = 128
NUM_LABELS = len(LABEL_COLS)
LEARNING_RATE = 1e-5
EPOCHS = 4 # Total epochs: 2 (initial run) + 4 (requested) = 6
BATCH_SIZE = 16

# --- 1. Load and Prepare Data ---
df = pd.read_csv("hate_speech_hindi_final.csv")
y = df[LABEL_COLS].values
X_train, X_val, y_train, y_val = train_test_split(
    df[TEXT_COL].tolist(),
    y,
    test_size=0.2,
    random_state=42
)

# --- 2. Calculate Sample Weights ---
pos_counts = y_train.sum(axis=0)
neg_counts = len(y_train) - pos_counts
total = pos_counts + neg_counts
pos_weights = (1 / pos_counts) * (total / 2.0)
neg_weights = (1 / neg_counts) * (total / 2.0)

sample_weight_matrix_5d = np.zeros_like(y_train, dtype=np.float32)
for i in range(NUM_LABELS):
    sample_weight_matrix_5d[:, i][y_train[:, i] == 1] = pos_weights[i]
    sample_weight_matrix_5d[:, i][y_train[:, i] == 0] = neg_weights[i]

sample_weights_1d = sample_weight_matrix_5d.mean(axis=1)


# --- 3. Load Tokenizer and Model ---
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# Re-initializing the model for a full 7-epoch run
model = TFAutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    problem_type="multi_label_classification"
)


# --- 4. Tokenization and Dataset Creation ---
train_encodings = tokenizer(
    X_train,
    truncation=True,
    padding=True,
    max_length=MAX_LENGTH,
    return_tensors="tf"
)
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train,
    sample_weights_1d
)).shuffle(100).batch(BATCH_SIZE)

val_encodings = tokenizer(
    X_val,
    truncation=True,
    padding=True,
    max_length=MAX_LENGTH,
    return_tensors="tf"
)
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_val
)).batch(BATCH_SIZE)


# --- 5. Compile and Train the Model (7 Epochs with Per-Sample Weighting) ---
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

print("-" * 50)
print(f"--- Starting NEW Training Run for TOTAL of {EPOCHS} Epochs with Per-Sample Averaged Weighting ---")
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset
)
print("--- Training Complete ---")
print("-" * 50)


# --- 6. Prediction and Per-Class Optimized Evaluation ---
print("--- Generating Predictions on Validation Data ---")
pred_outputs = model.predict(val_dataset)
logits = pred_outputs.logits
probs = tf.sigmoid(logits).numpy()
y_true = y_val

# Threshold Optimization
print("\n## 🧪 Threshold Optimization...")
best_thresholds = {}

for i, col_name in enumerate(LABEL_COLS):
    col_probs = probs[:, i]
    col_y_true = y_true[:, i]
    best_f1 = -1
    best_t = 0.5

    for t in np.arange(0.01, 0.61, 0.02):
        y_pred_t = (col_probs >= t).astype(int)
        f1 = f1_score(col_y_true, y_pred_t, zero_division=0)

        if f1 > best_f1:
            best_f1 = f1
            best_t = t

    best_thresholds[col_name] = best_t
    print(f"Class '{col_name}': Optimal Threshold = {best_t:.2f} (F1: {best_f1:.4f})")

# Final Metrics
y_pred_final_multi = np.zeros_like(y_true, dtype=int)
for i, col_name in enumerate(LABEL_COLS):
    t = best_thresholds[col_name]
    y_pred_final_multi[:, i] = (probs[:, i] >= t).astype(int)

subset_accuracy = accuracy_score(y_true, y_pred_final_multi)
macro_f1_final = f1_score(y_true, y_pred_final_multi, average="macro", zero_division=0)
micro_f1_final = f1_score(y_true, y_pred_final_multi, average="micro", zero_division=0)

print("\n--- Final Evaluation with Per-Class Optimized Thresholds ---")
print(f"Exact Match Accuracy: {subset_accuracy:.4f}")
print(f"Macro F1 Score: {macro_f1_final:.4f}")
print(f"Micro F1 Score: {micro_f1_final:.4f}")

print("\nClassification Report:\n")
print(classification_report(
    y_true,
    y_pred_final_multi,
    target_names=LABEL_COLS,
    zero_division=0
))

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier', 'bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


--------------------------------------------------
--- Starting NEW Training Run for TOTAL of 4 Epochs with Per-Sample Averaged Weighting ---
Epoch 1/4
1994/1994 [==============================] - 946s 450ms/step - loss: 0.5786 - accuracy: 0.2629 - val_loss: 0.5069 - val_accuracy: 0.3971
Epoch 2/4
1994/1994 [==============================] - 876s 439ms/step - loss: 0.4883 - accuracy: 0.4219 - val_loss: 0.4460 - val_accuracy: 0.4944
Epoch 3/4
1994/1994 [==============================] - 876s 439ms/step - loss: 0.4270 - accuracy: 0.5587 - val_loss: 0.3876 - val_accuracy: 0.6435
Epoch 4/4
1994/1994 [==============================] - 875s 439ms/step - loss: 0.3551 - accuracy: 0.7003 - val_loss: 0.3398 - val_accuracy: 0.7458
--- Training Complete ---
--------------------------------------------------
--- Generating Predictions on Validation Data ---
499/499 [==============================] - 61s 118ms/step

## 🧪 Threshold Optimization...
Class 'defamation': Optimal Threshold = 0.57 (F1: 0.7